https://docs.python.org/zh-cn/3/library/concurrent.futures.html
    
concurrent.futures 模块提供异步执行可调用对象高层接口。

异步执行可以由 ThreadPoolExecutor 使用线程或由 ProcessPoolExecutor 使用单独的进程来实现。 两者都是实现抽像类 Executor 定义的接口。

# ThreadPoolExecutor

ThreadPoolExecutor 是 Executor 的子类，它使用线程池来异步执行调用。

当回调已关联了一个 Future 然后再等待另一个 Future 的结果时就会发产死锁情况。例如:

In [1]:
# 死锁例子1
import time
from concurrent.futures import ThreadPoolExecutor,as_completed

def wait_on_b():
    time.sleep(5)
    print(b.result())  # b will never complete because it is waiting on a.
    return 5

def wait_on_a():
    time.sleep(5)
    print(a.result())  # a will never complete because it is waiting on b.
    return 6


executor = ThreadPoolExecutor(max_workers=2)
a = executor.submit(wait_on_b)
b = executor.submit(wait_on_a)

In [2]:
# 死锁例子2
def wait_on_future():
    f = executor.submit(pow, 5, 2)
    # This will never complete because there is only one worker thread and
    # it is executing this function.
    print(f.result())

executor = ThreadPoolExecutor(max_workers=1)
executor.submit(wait_on_future)

<Future at 0x22ac7693ac0 state=running>

# ThreadPoolExecutor 例子

In [5]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://some-made-up-domain.com/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# We can use a with statement to ensure threads are cleaned up promptly
with ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

'http://www.bbc.co.uk/' generated an exception: [WinError 10054] An existing connection was forcibly closed by the remote host
'http://www.cnn.com/' generated an exception: <urlopen error [WinError 10054] An existing connection was forcibly closed by the remote host>
'http://europe.wsj.com/' generated an exception: [WinError 10054] An existing connection was forcibly closed by the remote host
'http://some-made-up-domain.com/' generated an exception: <urlopen error [Errno 11001] getaddrinfo failed>
'http://www.foxnews.com/' page is 283254 bytes


In [7]:
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed
values = [2,3,4,5]
def square(n):
   return n * n

def main():
   with ThreadPoolExecutor(max_workers = 3) as executor:
      results = executor.map(square, values)
        
   for result in results:
      print(result)
        
if __name__ == '__main__':
   main()

4
9
16
25


In [8]:
from concurrent.futures import ThreadPoolExecutor
from time import sleep
def task(message):
   sleep(2)
   return message

def main():
   executor = ThreadPoolExecutor(5)
   future = executor.submit(task, ("Completed"))
   print(future.done())
   sleep(2)
   print(future.done())
   print(future.result())
    
if __name__ == '__main__':
    main()

False
True
Completed
